# Comparing randomized search and grid search for hyperparameter estimation

Adapted from http://scikit-learn.org/stable/auto_examples/model_selection/randomized_search.html

Compare randomized search and grid search for optimizing hyperparameters of a random forest. All parameters that influence the learning are searched simultaneously (except for the number of estimators, which poses a time / quality tradeoff).

The randomized search and the grid search explore exactly the same space of parameters. The result in parameter settings is quite similar, while the run time for randomized search is drastically lower.

The performance is slightly worse for the randomized search, though this is most likely a noise effect and would not carry over to a held-out test set.

Note that in practice, one would not search over this many different parameters simultaneously using grid search, but pick only the ones deemed most important.

In [1]:
using ScikitLearn
using PyCall: @pyimport

@pyimport scipy.stats as stats
using ScikitLearn.GridSearch: GridSearchCV, RandomizedSearchCV

@sk_import datasets: load_digits
@sk_import ensemble: RandomForestClassifier

In [2]:
# get some data
digits = load_digits()
X, y = digits["data"], digits["target"]

# build a classifier
clf = RandomForestClassifier(n_estimators=20)

# Utility function to report best scores
function report(grid_scores, n_top=3)
    top_scores = sort(grid_scores, by=x->x.mean_validation_score, rev=true)[1:n_top]
    for (i, score) in enumerate(top_scores)
        println("Model with rank:$i")
        @printf("Mean validation score: %.3f (std: %.3f)\n",
                score.mean_validation_score,
                std(score.cv_validation_scores))
        println("Parameters: $(score.parameters)")
        println("")
    end
end

# specify parameters and distributions to sample from
param_dist = Dict("max_depth"=> [3, nothing],
        "max_features"=> stats.randint(1, 11),
        "min_samples_split"=> stats.randint(1, 11),
        "min_samples_leaf"=> stats.randint(1, 11),
        "bootstrap"=> [true, false],
              "criterion"=> ["gini", "entropy"])

# run randomized search
n_iter_search = 20
random_search = RandomizedSearchCV(clf, param_dist,
                                   n_iter=n_iter_search)

start = time()
fit!(random_search, X, y)
@printf("RandomizedSearchCV took %.2f seconds for %d candidates, parameter settings.\n", (time() - start), n_iter_search)

report(random_search.grid_scores_)

# use a full grid over all parameters
param_grid = Dict("max_depth"=> [3, nothing],
              "max_features"=> [1, 3, 10],
              "min_samples_split"=> [1, 3, 10],
              "min_samples_leaf"=> [1, 3, 10],
              "bootstrap"=>[true, false],
              "criterion"=> ["gini", "entropy"])

# run grid search
grid_search = GridSearchCV(clf, param_grid)

start = time()
fit!(grid_search, X, y)

@printf("GridSearchCV took %.2f seconds for %d candidate parameter settings.\n",
time() - start, length(grid_search.grid_scores_))

report(grid_search.grid_scores_)

RandomizedSearchCV took 9.26 seconds for 20 candidates, parameter settings.
Model with rank:1
Mean validation score: 0.923 (std: 0.003)
Parameters: Dict{Symbol,Any}(:max_features=>5,:bootstrap=>true,:min_samples_split=>6,:max_depth=>nothing,:criterion=>"entropy",:min_samples_leaf=>2)

Model with rank:2
Mean validation score: 0.922 (std: 0.023)
Parameters: Dict{Symbol,Any}(:max_features=>10,:bootstrap=>false,:min_samples_split=>3,:max_depth=>nothing,:criterion=>"gini",:min_samples_leaf=>3)

Model with rank:3
Mean validation score: 0.912 (std: 0.026)
Parameters: Dict{Symbol,Any}(:max_features=>8,:bootstrap=>true,:min_samples_split=>6,:max_depth=>nothing,:criterion=>"entropy",:min_samples_leaf=>5)

GridSearchCV took 37.70 seconds for 216 candidate parameter settings.
Model with rank:1
Mean validation score: 0.934 (std: 0.019)
Parameters: Dict{Symbol,Any}(:max_features=>10,:bootstrap=>false,:min_samples_split=>1,:max_depth=>nothing,:min_samples_leaf=>1,:criterion=>"gini")

Model with rank: